In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
test_images = test_images.reshape(len(test_images), 28, 28, 1)
train_images = train_images.reshape(len(train_images), 28, 28, 1)

In [ ]:
def build_model(hp):
  model = keras.Sequential(
      [
       keras.layers.Conv2D(
           filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
           kernel_size = hp.Choice('conv_1_kernel', values= [3,5]),
           activation = 'relu',
           input_shape = (28, 28, 1)
       ),
       keras.layers.Conv2D(
           filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step = 16),
           kernel_size = hp.Choice('conv_2_kernel', values= [3,5]),
           activation = 'relu',
       ),
       keras.layers.Flatten(),
       keras.layers.Dense(
           units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
           activation = 'relu'
       )
      ])
  model.compile( optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])), loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'])
  return model


In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5, directory = 'output', project_name = 'Mnist Fashion')

In [ ]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)


Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |96                |64                
conv_1_kernel     |5                 |5                 
conv_2_filter     |64                |48                
conv_2_kernel     |3                 |5                 
dense_1_units     |128               |48                
learning_rate     |0.01              |0.001             

Epoch 1/3
1688/1688 [==============================] - 248s 147ms/step - loss: 14.4745 - accuracy: 0.1001 - val_loss: 14.5305 - val_accuracy: 0.0985
Epoch 2/3
1688/1688 [==============================] - 247s 147ms/step - loss: 14.5037 - accuracy: 0.1002 - val_loss: 14.5305 - val_accuracy: 0.0985
Epoch 3/3
1688/1688 [==============================] - 248s 147ms/step - loss: 14.5037 - accuracy: 0.1002 - val_loss: 14.5305 - val_accuracy: 0.0985


In [ ]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        76832     
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               1433712   
Total params: 1,513,040
Trainable params: 1,513,040
Non-trainable params: 0
_________________________________________________________________
